2024-02-19 13:34:20.487917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 13:34:20.487944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 13:34:20.489110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 13:34:20.495489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-19 13:34:21.189766: W tensorflow/compiler/tf2

In [2]:
import tensorflow as tf
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from utils.helper import *
from utils.constants import *
from inception.main import build_model

df = read_dataset("ArrowHead")

X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)
# apply one-hot encoder
y_train=to_categorical(df[1])
y_test=to_categorical(df[3])

model = build_model(input_shape=X_train.shape[1:] , num_classes=y_train.shape[1], num_modules=1)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 251, 1)]             0         []                            
                                                                                                  
 conv1d_16 (Conv1D)          (None, 251, 32)              32        ['input_2[0][0]']             
                                                                                                  
 max_pooling1d_3 (MaxPoolin  (None, 251, 1)               0         ['input_2[0][0]']             
 g1D)                                                                                             
                                                                                                  
 conv1d_17 (Conv1D)          (None, 251, 32)              10240     ['conv1d_16[0][0]']     

In [2]:
num_modules=2
results=[]
histories=[]

try:
    for dataset in  ['ArrowHead', 'Wine', 'OliveOil', 'Car', 'BeetleFly']:
        with tf.device("/device:GPU:0"):
            # Load data
            df = read_dataset(dataset)

            X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
            X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)

            # apply one-hot encoder
            y_train=to_categorical(df[1])
            y_test=to_categorical(df[3])
            print("test",X_train.shape)

            # Compile the model
            model = build_model(input_shape=X_train.shape[1:] , num_classes=y_train.shape[1], num_modules=num_modules)

            callbacks =[
                tf.keras.callbacks.ModelCheckpoint(
                    f"best_models/best_model_{dataset}.tf", save_weights_only=True, monitor="lr"
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="lr", factor=0.5, patience=50, min_lr=0.0001
                ),
            ]

            # Train the model
            history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data= (df[2], y_test), callbacks = callbacks)

            # Evaluate the model
            train_loss, train_accuracy = model.evaluate(X_train, y_train)
            test_loss, test_accuracy = model.evaluate(X_test, y_test)

            results.append([dataset, train_loss, train_accuracy, test_loss, test_accuracy])
            histories.append(history.history)
            
    pd.DataFrame(results, columns=["Dataset","Train loss", "Train accuracy","Test loss", "Test accuracy"]).to_csv(f"result_{num_modules}.csv", index=False)

except Exception as e:
    print("An error occurred:", e)


for i in range(len(histories)):
    plot_loss(histories[i], UNIVARIATE_DATASET_NAMES_2018[i], num_modules)

test (36, 251, 1)
Epoch 1/1000


2024-02-19 14:46:09.612530: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-02-19 14:46:11.468612: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc928b4ebc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-19 14:46:11.468641: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P400, Compute Capability 6.1
2024-02-19 14:46:11.474782: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708350371.545276  164649 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/3 [=========>....................] - ETA: 11s - loss: 1.3188 - accuracy: 0.1875

2024-02-19 14:46:14.727494: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-19 14:46:14.833009: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3/3 [==============================] - 9s 2s/step - loss: 1.5706 - accuracy: 0.1389 - val_loss: 2.4417 - val_accuracy: 0.3029 - lr: 0.0100
Epoch 2/1000
3/3 [==============================] - 0s 101ms/step - loss: 0.9625 - accuracy: 0.5278 - val_loss: 6.7559 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 3/1000
3/3 [==============================] - 0s 111ms/step - loss: 0.8723 - accuracy: 0.6111 - val_loss: 10.6507 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 4/1000
3/3 [==============================] - 0s 107ms/step - loss: 0.8228 - accuracy: 0.5278 - val_loss: 8.5932 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 5/1000
3/3 [==============================] - 0s 99ms/step - loss: 0.7780 - accuracy: 0.5833 - val_loss: 6.1948 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 6/1000
3/3 [==============================] - 0s 120ms/step - loss: 0.6828 - accuracy: 0.6667 - val_loss: 7.4631 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 7/1000
3/3 [==============================] - 0s 95ms/step - loss: 0.7181 - accu

2024-02-19 14:50:32.530964: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.60GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-19 14:50:32.638510: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.60GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


4/4 [==============================] - 8s 1s/step - loss: 0.9827 - accuracy: 0.2807 - val_loss: 6.6890 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 2/1000
1/4 [======>.......................] - ETA: 0s - loss: 0.7952 - accuracy: 0.3750

2024-02-19 14:50:34.850981: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-19 14:50:34.963654: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


4/4 [==============================] - 0s 54ms/step - loss: 0.7430 - accuracy: 0.4035 - val_loss: 4.1900 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 3/1000
4/4 [==============================] - 0s 53ms/step - loss: 0.7242 - accuracy: 0.4737 - val_loss: 3.4092 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 4/1000
4/4 [==============================] - 0s 51ms/step - loss: 0.7235 - accuracy: 0.5263 - val_loss: 5.5466 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 5/1000
4/4 [==============================] - 0s 54ms/step - loss: 0.7259 - accuracy: 0.5263 - val_loss: 2.3899 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 6/1000
4/4 [==============================] - 0s 53ms/step - loss: 0.6976 - accuracy: 0.4561 - val_loss: 1.2270 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 7/1000
4/4 [==============================] - 0s 51ms/step - loss: 0.6933 - accuracy: 0.5263 - val_loss: 1.5467 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 8/1000
4/4 [==============================] - 0s 53ms/step - loss: 0.6944 - accurac

In [3]:
for i in range(len(histories)):
    plot_loss(histories[i], UNIVARIATE_DATASET_NAMES_2018[i])

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>